In [1]:
!nvidia-smi

Mon Dec 11 10:30:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install transformers "transformers[torch]" torch torch-vision datasets sacremoses accelerate evaluate pandas

In [6]:
!unzip dumped-dataset.zip

Archive:  dumped-dataset.zip
   creating: dumped-dataset/
  inflating: dumped-dataset/.DS_Store  
  inflating: __MACOSX/dumped-dataset/._.DS_Store  
  inflating: dumped-dataset/dataset_dict.json  
   creating: dumped-dataset/test/
   creating: dumped-dataset/train/
   creating: dumped-dataset/validation/
  inflating: dumped-dataset/test/state.json  
  inflating: dumped-dataset/test/dataset_info.json  
  inflating: dumped-dataset/test/data-00000-of-00001.arrow  
  inflating: dumped-dataset/train/state.json  
  inflating: dumped-dataset/train/dataset_info.json  
  inflating: dumped-dataset/train/data-00000-of-00001.arrow  
  inflating: dumped-dataset/validation/state.json  
  inflating: dumped-dataset/validation/dataset_info.json  
  inflating: dumped-dataset/validation/data-00000-of-00001.arrow  


In [7]:
import transformers
import datasets as ds

datasets = ds.DatasetDict.load_from_disk('./dumped-dataset')
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 84990
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10230
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 7424
    })
})

In [8]:
herbert_model_name = 'allegro/herbert-base-cased'



In [9]:
from transformers import AutoTokenizer

pl_tokenizer = AutoTokenizer.from_pretrained(herbert_model_name)

def tokenize_fn(sample):
    return pl_tokenizer(sample['text'], padding='max_length', truncation=True)

tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [13]:
tokenized_datasets = datasets.map(tokenize_fn, batched=True)
tokenized_datasets['train']

Map:   0%|          | 0/84990 [00:00<?, ? examples/s]

Map:   0%|          | 0/10230 [00:00<?, ? examples/s]

Map:   0%|          | 0/7424 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 84990
})

In [14]:

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(herbert_model_name, num_labels=2)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [23]:
from transformers import TrainingArguments
import numpy as np

arguments = TrainingArguments(
    output_dir="./model/",
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-05,
    num_train_epochs=1,
    logging_first_step=True,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="epoch",
    fp16=False,
)

In [24]:
import evaluate

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return metric.compute(predictions=preds, references=labels)

In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_datasets['train'].shuffle(seed=84),
    eval_dataset=tokenized_datasets['test'].shuffle(seed=84),
    compute_metrics=compute_metrics
)

In [26]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
50,0.468700,0.450654,0.833236
100,0.468400,0.464943,0.833236
150,0.444600,0.451503,0.833236
200,0.457800,0.454203,0.833236


KeyboardInterrupt: ignored

Robiłem kilka podejść do trenowania, z różnymi parametrami, ale `accuracy` nie drgnęło ani razu, nawet po 1,5h (1900/5312 kroków) treningu. Muszę przyznać, że nie wiem dlaczego, ale nie widzę sensu zostawiania trenowania na 3-4h w momencie gdy model w żaden sposób nie poprawia swojego accuracy...